# Outputs

Parameters are typically used for the _inputs_ to a Parameterized objects, declaring as precisely as possible which inputs are allowed. You can also declare what _outputs_ are generated by a Parameterized, using `@param.output`. At present, Param itself does not use output declarations internally, but they can be queried by Param-based programs to allow safe chaining of Parameterized objects into pipelines, such as for the [boxflow](https://github.com/ioam/boxflow) dataflow programming system or the multi-page pipelines in [Panel](https://panel.holoviz.org/user_guide/Pipelines.html). 

`@param.output` annotates a method  on a Parameterized class to declare that it returns one or more outputs of a specified type. As a simple example, this declaration indicates that the given function returns a number:

In [ ]:
import param

class P(param.Parameterized):
    a = param.Number(default=5, bounds=(0, 10))
    b = param.Number(default=5, bounds=(0, 10))

    @param.output(param.Number)
    def product(self):
        return self.a * self.b
    
p = P(a=2, b=3)
p.product()

If a program wants to know if the output from object `p` is suitable for connecting to an input of some other object `q`, it can query this output type:

In [ ]:
p.param.outputs()

This return value is of the form _name_: (_type_, _method_, _index_), which here indicates that:
- object `p` generates one output called `product` 
- `product` is of type `param.Number`
- `product` can be generated by invoking method `p.product()` 
- `product` will be returned directly as a single value by the method (indicated by an index of `None`; otherwise the index would indicate the position of this particular output in a tuple returned by the method)

An automated program could use this information to decide whether the output from one Parameterized is suitable for connecting to the input of another.

The above example is typical, but let's review the other output declarations accepted by `@param.output`. The simplest declaration declares the method returns an object of the same name as the method, without any type guarantees:

```python
@param.output()
def product(self): return self.a * self.b
```

More commonly, a parameter type will be specified as above, indicating that this method returns a value of that type, again defaulting to the method name:

```python
@param.output(param.Number())
def product2(self): return self.a * self.b
```

The output name can be declared explicitly as a keyword argument if desired, e.g. if the method name is not a suitable output name:

```python
@param.output(result=param.Number())
def __call__(self): return self.a * self.b
```

Multiple outputs may be declared using keywords mapping from output name to the type (Python >= 3.6 only):

```python
@param.output(prod_num=param.Number(), prod_str=param.String())
def products(self): 
    prod = self.a * self.b
    return prod, str(prod)
```

`@param.output` also accepts Python object types, which will be upgraded to a ClassSelector:

```python
@param.output(int)
def int_product(self): return int(self.a * self.b)
```

We can see these various options in action:

In [ ]:
class Q(param.Parameterized):
    a = param.Number(default=5, bounds=(0, 10))
    b = param.Number(default=5, bounds=(0, 10))

    @param.output()
    def product(self): return self.a * self.b

    @param.output(param.Number())
    def product2(self): return self.a * self.b

    @param.output(result=param.Number())
    def __call__(self): return self.a * self.b

    @param.output(prod_num=param.Number(), prod_str=param.String())
    def products(self): 
        prod = self.a * self.b
        return prod, str(prod)

    @param.output(int)
    def int_product(self): return int(self.a * self.b)

q=Q()
q

In [ ]:
q.param.outputs()

Here, you can see that there are _two_ outputs from `prod_str()`, one of type Number and one of type String, and that they are in the order (number, string) in the tuple. The other outputs are all a single result returned directly from that method, with the indicated types (defaulting to `Parameter`) and names. Annotating outputs in this way can help you build large, flexible systems for connecting Parameterized objects together into larger data or computational structures.